### Content:
- Generate cooccurrences (value represents number of articles in which two companies occur together)

#### TODO:
- Other measures for cooccurrence?

In [1]:
import os
import re
import glob
from datetime import datetime
import sys
sys.path.append("..") # Adds higher directory to python modules path for importing from src dir

import pandas as pd
import numpy as np
import tqdm
import matplotlib
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook as tqdm

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

%matplotlib inline
%load_ext autotime
%load_ext autoreload
%autoreload 2

c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
c:\users\thomas\hpi\ma\venv\lib\importlib\_bootstrap.py:205: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


### Get companies

In [3]:
from src.datasets import NyseSecuritiesDataset
from src.datasets import NyseStocksDataset
import src.nlp_utils as nlp_utils

time: 853 ms


In [4]:
stocks_ds = NyseStocksDataset(file_path='../data/nyse/prices-split-adjusted.csv')
securities_ds = NyseSecuritiesDataset(file_path='../data/nyse/securities.csv')
companies = securities_ds.get_all_company_names()  # List[Tuple[symbol, name]]
stocks_ds.load()


time: 9.39 s


### Calculate Cooccurrences

In [ ]:
def get_cooccurrences(occs):
    grouped = occs.groupby(['article_id', 'stock_symbol'], sort=False)
    occ_per_article = grouped.size().reset_index().pivot('article_id', 'stock_symbol')\
        .fillna(0).astype(int)
    occ_per_article.columns = occ_per_article.columns.droplevel(0)
    comp_does_occure = (occ_per_article != 0).astype(int)
    cooc = comp_does_occure.T.dot(comp_does_occure)
    # comps_article_counts = np.diag(cooc)  # (occ_per_article != 0).astype(int).sum(axis=0)
    np.fill_diagonal(cooc.values, 0)
    cooc.index.levels[0].name = None
    cooc.index.levels[1].name = None
    return cooc

In [201]:
occs_b = pd.read_csv('occurrences-bloomberg-v2.csv', index_col=0)
occs_r = pd.read_csv('occurrences-reuters-v2.csv', index_col=0)
occs = pd.concat([occs_b, occs_r]).reset_index(drop=True)
coocs = nlp_utils.get_cooccurrences(occs)
# coocs.to_csv('cooccurrences.csv')
# readable_coocurrences(coocs)

time: 19.6 s
